In [3]:
import os
import json
import openai
from tqdm import tqdm
import polars as pl
from dotenv import dotenv_values

config = dotenv_values("../.env") 

In [46]:
openai.api_type = "azure"
openai.api_base = config["OPENAI_ENDPOINT"]
openai.api_version = "2023-05-15"
openai.api_key = config["OPENAI_KEY"]

response = openai.ChatCompletion.create(
    engine=config["OPENAI_ENGINE"], 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response)
print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-7rtY0pTDqMJlRQ9fNFdrC8qdiUoGL",
  "object": "chat.completion",
  "created": 1693079264,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Yes, several Azure AI services support Customer Managed Keys (CMK). The support can vary depending on specific services, so it's important to double-check the specific documentation of the service you're interested in. Services such as Azure Cognitive Services, Azure Search, and Azure Machine Learning are ones that support CMK, to name a few.\n"
      }
    }
  ],
  "usage": {
    "completion_tokens": 67,
    "prompt_tokens": 55,
    "total_tokens": 122
  }
}
Yes, several Azure AI services support Customer Managed Keys (CMK). The support can vary depending on specific services, so it's important to double-check the specific documentation of the service you're interested in. Services such as Azure Cognitive Services, Azur

In [40]:
data = pl.read_json("../data/issues_v1.json").filter(pl.col("number").is_not_null()).sort("number")
title, body = data.select("title", "body").row(1500)
title, body

('custom f(x) in heatmap plot',
 'In some cases it is more interesting to see not exact f(x) value but transformed one, or the value of another sample parameter that is correlated with f(x), for this reason it would be nice to have a lambda of Sample -> value for the heatmap plot to be able to have custom stuff there.')

In [41]:
response = openai.ChatCompletion.create(
    engine=config["OPENAI_ENGINE"], 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
        {"role": "user", "content": f"Is the description of issue good? \ntitle: {title}\nbody: {body}\nAnswer with Yes/No"},
    ]
)

print(response['choices'][0]['message']['content'])

Yes


In [42]:
response = openai.ChatCompletion.create(
    engine=config["OPENAI_ENGINE"], 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
        {"role": "user", "content": f"Give me one sentence summary of following issue, use ten words or less: \ntitle: {title}\nbody: {body}"},
    ]
)

print(response['choices'][0]['message']['content'])

Requesting feature for custom function application in heatmap plotting.


In [43]:
response = openai.ChatCompletion.create(
    engine=config["OPENAI_ENGINE"], 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
        {"role": "user", "content": f"Suggest short label for this issue. Label should be of maximum two words: \ntitle: {title}\nbody: {body}"},
    ]
)

print(response['choices'][0]['message']['content'])

"Custom Plotting"


In [45]:
issue_labels = [
    "bug",
    "documentation",
    "enhancement",
    "good first issue",
    "stale"
]


response = openai.ChatCompletion.create(
    engine=config["OPENAI_ENGINE"], 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
        {"role": "user", "content": f"Select label from {issue_labels}, where stale label is for issues that are too short and not clear: \ntitle: {title}\nbody: {body}\nAnswer with label name"},
    ]
)

print(response['choices'][0]['message']['content'])

'enhancement'
